In [1]:
from spotify import Client, OAuth
import pandas as pd

In [2]:
auth = OAuth('f716b10140c04e9997c89e1d094745f6', 'b9aa1c134fe244ebaed7b3cba541f6a7')
auth.request_client_credentials()

In [4]:
spotifyClient = Client(auth)
spotifyAPI = spotifyClient.api

In [26]:
def getTrackAttributes(trackIdList = []):
    print(trackIdList)
    trackJson = spotifyAPI.tracks_audio_features(trackIdList)
    trackList = trackJson['audio_features']
    trackList = filter(None, trackList)
    trackList = [x for x in trackList if x is not None]
    return trackList

In [27]:
def  getAttributeDF(trackFeatList):
    trackDF = pd.DataFrame(trackFeatList)
    trackDF = trackDF.drop(['analysis_url', 'track_href', 'type', 'uri'], axis=1)
    return trackDF

In [15]:
def getPlaylistTrack(play_id,user_id):
    tracks = spotifyAPI.user_playlist_tracks(playlist_id= play_id, user_id= user_id,fields='items(track.id,track.album.name,track.name,track.artists)')
    #tracks = list(filter(None, tracks))
    return tracks

In [16]:
def trackJSONtoDF(tracks,genre):
    flatData = []

    for i in range(len(tracks['items'])):
        trackObj = {}
        
        trackObj['id'] = tracks['items'][i]['track']['id']
        trackObj['album'] = tracks['items'][i]['track']['album']['name']
        trackObj['track_name'] = tracks['items'][i]['track']['name']
        trackObj['artist'] = tracks['items'][i]['track']['artists'][0]['name']
        trackObj['genre'] = genre
        if trackObj['id'] != None:
            flatData.append(trackObj)
    return pd.DataFrame(flatData)

In [28]:
def pipeLineFunc(play_id,user_id,genre):
    # Fetch playlist tracks
    plTracks = getPlaylistTrack(play_id,user_id)
    
    
    # Convert to DF
    plTracksDF =  trackJSONtoDF(plTracks,genre)

    # Get list of track ids
    trackIds = list(plTracksDF['id'])

    # Get tracks features
    trackFeat = getTrackAttributes(trackIds)

    # Convert JSON to DF
    trackFeatDF = getAttributeDF(trackFeat)

    # Merging 2 data frames
    resultDF = pd.concat([plTracksDF, trackFeatDF], axis=1, join='inner')

    # Droping duplicate ID columns
    resultDF = resultDF.T.drop_duplicates().T

    return resultDF

In [20]:
playListData = [{'user':'spotify','playlist_id':'37i9dQZF1DWWwzidNQX6jx','genre':'70sRock'},
{'user':'spotify','playlist_id':'37i9dQZF1DX153gOfbCM2i','genre':'2000sRock'}]

In [29]:
playlistDF = pd.DataFrame(playListData)
print(playlistDF)

       genre             playlist_id     user
0    70sRock  37i9dQZF1DWWwzidNQX6jx  spotify
1  2000sRock  37i9dQZF1DX153gOfbCM2i  spotify


In [31]:
trackDFMain = pd.DataFrame()
for i in range(len(playlistDF)):
    resDF = pipeLineFunc(playlistDF["playlist_id"][i],playlistDF["user"][i],playlistDF["genre"][i])
    trackDFMain = trackDFMain.append(resDF)

    

['2GAIycsMaDVtMtdvxzR2xI', '5CQ30WqJwcep0pYcV4AMNc', '40riOy7x9W7GXjyGp4pjAv', '5jzma6gCzYtKB1DbEwFZKH', '14XWXWv5FoCbFzLksawpEe', '6KTv0Z8BmVqM7DPxbGzpVC', '02DurCgOvDdX0uKEjqcl3W', '0LrwgdLsFaWh9VXIjBRe8t', '3Bh6uInhcVBVvLraGZdkKD', '1IqFh00G2kvvMm8pRMpehA', '52dm9op3rbfAkc1LGXgipW', '7pKfPomDEeI4TPT6EOYjn9', '7iN1s7xHE4ifF5povM6A48', '5MxNLUsfh7uzROypsoO5qe', '0SYRVn2YF7HBscQEmlkpTI', '2zYzyRzz6pRmhPzyfMEC8s', '6FsQrRpBLgsrFeAeiQqytm', '5SAUIWdZ04OxYfJFDchC7S', '4YMQXzscifAREG0a7KNGhB', '5xD021bM8VbqrhCVhxRozx', '5EWPGh7jbTNO2wakv8LjUI', '3ykSdTGmYPFl8pDBXer1zG', '0UAJH0k4k3slcE83a9UGCe', '3qT4bUD1MaWpGrTwcvguhb', '4KcH1ZRV2W1q7Flq0QqC76', '1JkZg3eMQTmTn93E8Yd3UL', '6PmckFG2BhEELc5OM7O7DS', '7kNNylJ1kswWbHeRM6UDuE', '7MRyJPksH3G2cXHN8UKYzP', '05uGBKRCuePsf43Hfm0JwX', '5HNCy40Ni5BZJFw1TKzRsC', '6hTcuIQa0sxrrByu9wTD7s', '2kkvB3RNRzwjFdGhaUA0tz', '4alHo6RGd0D3OUbTPExTHN', '3LoDeIbiR12sAznmpNEmKA', '2QgWuCtBpNIpl5trmKCxRf', '5UuikgHTxSRFRnC0zXx10i', '0Rp4akmfqBTpbDM6EoqtNy', '3IOQZRcEkp

,album,artist,genre,id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Queen 40 Limited Edition Collector's Box Set,Queen,70sRock,2GAIycsMaDVtMtdvxzR2xI,Bohemian Rhapsody - Remastered 2011,0.254,0.397,354320,0.396,0,3,0.268,-9.91,1,0.05,142.913,4,0.227
1,Led Zeppelin IV (Deluxe Edition),Led Zeppelin,70sRock,5CQ30WqJwcep0pYcV4AMNc,Stairway To Heaven,0.58,0.338,482830,0.34,0.0032,9,0.116,-12.049,0,0.0339,82.433,4,0.197
2,Hotel California (Remastered),Eagles,70sRock,40riOy7x9W7GXjyGp4pjAv,Hotel California - Remastered,0.00574,0.579,391376,0.508,0.000494,2,0.0575,-9.484,1,0.027,147.125,4,0.609
3,London Calling,The Clash,70sRock,5jzma6gCzYtKB1DbEwFZKH,London Calling - Remastered,0.123,0.651,200480,0.801,0,0,0.0825,-7.34,1,0.0513,133.763,4,0.776
4,L.A. Woman,The Doors,70sRock,14XWXWv5FoCbFzLksawpEe,Riders On The Storm,0.25,0.549,434720,0.722,0.0138,11,0.11,-11.729,0,0.0283,103.885,4,0.716
5,Dressed To Kill,KISS,70sRock,6KTv0Z8BmVqM7DPxbGzpVC,Rock And Roll All Nite,0.0551,0.657,168827,0.932,0,1,0.047,-5.819,1,0.0992,144.751,4,0.888
6,Who Are You,The Who,70sRock,02DurCgOvDdX0uKEjqcl3W,Who Are You,0.299,0.653,381627,0.631,0.00265,9,0.0627,-11.334,1,0.0461,156.32,4,0.516
7,Hunky Dory (2015 Remastered Version),David Bowie,70sRock,0LrwgdLsFaWh9VXIjBRe8t,Changes - 2015 Remastered Version,0.505,0.547,217747,0.308,0,0,0.109,-11.75,1,0.0377,114.719,4,0.41
8,Cosmo's Factory (40th Anniversary Edition),Creedence Clearwater Revival,70sRock,3Bh6uInhcVBVvLraGZdkKD,Who'll Stop The Rain,0.0428,0.602,147493,0.801,0.000409,7,0.0739,-6.763,1,0.0344,123.562,4,0.576
9,Pearl (Legacy Edition),Janis Joplin,70sRock,1IqFh00G2kvvMm8pRMpehA,Me and Bobby McGee,0.302,0.453,271333,0.464,5.96e-05,2,0.152,-10.96,1,0.0431,93.19,4,0.658


In [32]:
uniquetrackDF = trackDFMain.drop_duplicates('id')

In [33]:
uniquetrackDF.to_csv('RockMusicInfo.csv')